In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import json
import pandas as pd

from helper_funcs import get_api_key_url, read_txt_file_to_string
API_KEY, API_URL = get_api_key_url(client="OPENAI")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=API_KEY) 
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

doc1 = read_txt_file_to_string("documents/doc1.txt")
doc2 = read_txt_file_to_string("documents/doc2.txt")

prompt_template = ChatPromptTemplate.from_template("""
Je bent een compliance-expert gespecialiseerd in DORA, ISO27001 en vergelijkbare standaarden.
Vergelijk de twee onderstaande documenten per criterium.
Geef per criterium:
- een samenvatting van hoe elk document het onderwerp behandelt
- een beoordeling van de mate van overeenstemming (bijv. "Volledig in lijn", "Gedeeltelijk", "Niet in lijn")
- een korte toelichting
- citeer indien mogelijk relevante fragmenten uit beide documenten met het pagina nummer erbij

Document A:
\"\"\"{doc1}\"\"\"

Document B:
\"\"\"{doc2}\"\"\"

Criteria:
{criteria}

Antwoord in geldig JSON-formaat als volgt:
[
    {{
    "criterium": "...",
    "overeenstemming": "...",
    "analyse_document_A": "...",
    "analyse_document_B": "...",
    "toelichting": "...",
    "referentie_A": "...",
    "referentie_B": "..."
    }},
    ...
]
""")

criteria = \
    "Governance structuur\n" \
    "Risicomanagement\n" \
    "Incident response\n" \
    "Outsourcing beleid"

chain = prompt_template | llm | StrOutputParser()

output = chain.invoke({
    "doc1": doc1,
    "doc2": doc2,
    "criteria": json.dumps(criteria, ensure_ascii=False)
})

def convert_json_output_to_table(output, print_as_md=True, save_as_csv=False, filename="vergelijkingstabel.csv"):
    """Converteert JSON-output van LLM naar een pandas DataFrame."""
    
    # --- JSON veilig parsen ---
    try:
        resultaten = json.loads(output)
    except json.JSONDecodeError:
        resultaten = json.loads(output[output.find('['):output.rfind(']')+1])

    # --- Omzetten naar tabel ---
    rows = []
    for item in resultaten:
        rows.append({
            "Criterium": item.get("criterium", ""),
            "Overeenstemming": item.get("overeenstemming", ""),
            "Analyse Document A": item.get("analyse_document_A", ""),
            "Analyse Document B": item.get("analyse_document_B", ""),
            "Toelichting": item.get("toelichting", ""),
            "Referentie A": item.get("referentie_A", ""),
            "Referentie B": item.get("referentie_B", ""),
        })

    df = pd.DataFrame(rows)

    # --- Markdown-weergave ---
    if print_as_md:
        print("\n📊 Vergelijkingsoverzicht:\n")
        print(df.to_markdown(index=False))

    # --- CSV-export ---
    if save_as_csv:
        df.to_csv(filename, index=False)
        print(f"\n💾 Tabel opgeslagen als: {filename}")

    return df

# --- Uitvoeren ---
df = convert_json_output_to_table(output, print_as_md=True, save_as_csv=False)


📊 Vergelijkingsoverzicht:

| Criterium            | Overeenstemming   | Analyse Document A                                                                                                                                               | Analyse Document B                                                                                                                                              | Toelichting                                                                                                                                                    | Referentie A              | Referentie B             |
|:---------------------|:------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_openai import ChatOpenAI
import json
import pandas as pd

from helper_funcs import get_api_key_url, read_txt_file_to_string

# --- Setup ---
API_KEY, API_URL = get_api_key_url(client="OPENAI")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=API_KEY)
# embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# --- Documenten ---
doc1 = read_txt_file_to_string("documents/doc1.txt")
doc2 = read_txt_file_to_string("documents/doc2.txt")

# --- Criteria ---
criteria = (
    "Governance structuur\n"
    "Risicomanagement\n"
    "Incident response\n"
    "Outsourcing beleid"
)

table_description = {
    "Criterium": "Zoals gegeven",
    "Overeenstemming": "Mate van overeenstemming tussen document A en B, bijvoorbeeld 'Volledig in lijn', 'Gedeeltelijk', of 'Niet in lijn'.",
    "Analyse Document A": "Korte samenvatting van hoe document A het criterium behandelt.",
    "Analyse Document B": "Korte samenvatting van hoe document B het criterium behandelt.",
    "Toelichting": "Een korte uitleg van de overeenkomsten of verschillen tussen de documenten.",
    "Referentie A": "Een citaat of tekstfragment (met paginanummer indien mogelijk) uit document A.",
    "Referentie B": "Een citaat of tekstfragment (met paginanummer indien mogelijk) uit document B."
}

prompt_template = ChatPromptTemplate.from_template(f"""
Je bent een compliance-expert gespecialiseerd in DORA, ISO27001 en vergelijkbare standaarden.
Je vergelijkt twee documenten aan de hand van specifieke criteria.
Vul onderstaande tabelstructuur in met de juiste informatie op basis van de inhoud van de documenten.

Document A:
\"\"\"{{doc1}}\"\"\"

Document B:
\"\"\"{{doc2}}\"\"\"

Criteria:
{{criteria}}

Antwoord in geldig JSON-formaat als volgt:
[
    {{table_description}},
    ...
]
""")

# --- 3️⃣ Chain bouwen ---
chain = prompt_template | llm | StrOutputParser()

# --- 4️⃣ Uitvoeren ---
output = chain.invoke({
    "doc1": doc1,
    "doc2": doc2,
    "criteria": json.dumps(criteria, ensure_ascii=False),
    "table_description": table_description
})

# --- 5️⃣ Parse JSON naar tabel ---
def json_to_table(output, print_as_md=True, save_as_csv=False):
    try:
        resultaten = json.loads(output)
    except json.JSONDecodeError:
        resultaten = json.loads(output[output.find('['):output.rfind(']')+1])

    df = pd.DataFrame(resultaten)

    if print_as_md:
        print("\n📊 Vergelijkingsoverzicht:\n")
        print(df.to_markdown(index=False))
    if save_as_csv:
        df.to_csv("vergelijking.csv", index=False)
        print("💾 Tabel opgeslagen als vergelijking.csv")

    return df

df = json_to_table(output, print_as_md=True, save_as_csv=True)



📊 Vergelijkingsoverzicht:

| Criterium            | Overeenstemming   | Analyse Document A                                                                                                                                                            | Analyse Document B                                                                                                                                                               | Toelichting                                                                                                                                                  | Referentie A                                                                                                                    | Referentie B                                                                                              |
|:---------------------|:------------------|:----------------------------------------------------------------------------------------------------------------------------------

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import pandas as pd
import json

from helper_funcs import get_api_key_url, read_txt_file_to_string
API_KEY, API_URL = get_api_key_url(client="OPENAI")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, api_key=API_KEY)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# --- Documenten ---
doc1 = read_txt_file_to_string("documents/doc1.txt")
doc2 = read_txt_file_to_string("documents/doc2.txt")

# --- Rubricvragen ---
criteria = (
    "1. Vorm: Is de tekst helder, zakelijk en passend bij de doelgroep?\n"
    "2. Structuur: Heeft de tekst een logische opbouw met duidelijke alinea’s en goede overgangen?\n"
    "3. Inhoud: Zijn de hoofdgedachte en argumenten overtuigend en relevant?\n"
    "4. Stijl: Is de toon professioneel en consequent?\n"
    "5. Taalgebruik: Zijn spelling, grammatica en zinsbouw correct?"
)

# --- Prompt ---
prompt_template = ChatPromptTemplate.from_template("""
Je bent een ervaren docent Nederlands gespecialiseerd in zakelijke communicatie.
Je beoordeelt twee zakelijke teksten aan de hand van een vaste rubric met vragen over vorm, inhoud en structuur.

Beoordeel elk criterium afzonderlijk voor beide teksten.
Voor elke vraag uit de rubric geef je:
- een beoordeling per tekst (bijv. "goed", "voldoende", "onvoldoende")
- een korte toelichting
- citeer een relevant tekstfragment waarop de beoordeling gebaseerd is

Document A:
\"\"\"{doc1}\"\"\"

Document B:
\"\"\"{doc2}\"\"\"

Rubricvragen:
{criteria}

Antwoord in geldig JSON-formaat als volgt:
[
    {{
        "criterium": "...",
        "beoordeling_document_A": "...",
        "toelichting_A": "...",
        "referentie_A": "...",
        "beoordeling_document_B": "...",
        "toelichting_B": "...",
        "referentie_B": "..."
    }},
    ...
]
""")

chain = prompt_template | llm | StrOutputParser()

output = chain.invoke({
    "doc1": doc1,
    "doc2": doc2,
    "criteria": json.dumps(criteria, ensure_ascii=False)
})

# --- JSON-parsing ---
try:
    resultaten = json.loads(output)
except json.JSONDecodeError:
    resultaten = json.loads(output[output.find('['):output.rfind(']')+1])

# --- Data omzetten naar tabel ---
rows = []
for item in resultaten:
    rows.append({
        "Criterium": item["criterium"],
        "Document A": f"{item['beoordeling_document_A']}\n\n{item['toelichting_A']}\n\n> {item['referentie_A']}",
        "Document B": f"{item['beoordeling_document_B']}\n\n{item['toelichting_B']}\n\n> {item['referentie_B']}",
    })

df = pd.DataFrame(rows)

# --- Tabel tonen ---
print("\n📊 Beoordelingsoverzicht:\n")
print(df.to_markdown(index=False))



📊 Beoordelingsoverzicht:

| Criterium   | Document A                                                                                                                                                      | Document B                                                                                                                                |
|:------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------|
| Vorm        | goed                                                                                                                                                            | voldoende                                                                                                                                 |
|             |    